In [45]:
import pandas as pd

emails = pd.read_csv('data/emails.csv',
                     header=0,
                     names=['emailsOpened', 'member', 'week'],
                     parse_dates=['week'])

emails.head()



,emailsOpened,member,week
0,3.0,1.0,2015-06-29
1,2.0,1.0,2015-07-13
2,2.0,1.0,2015-07-20
3,3.0,1.0,2015-07-27
4,1.0,1.0,2015-08-03


In [46]:
donations = pd.read_csv('data/donations.csv', parse_dates=['timestamp'])
donations.columns = ['amount', 'timestamp', 'member']
donations




,amount,timestamp,member
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0
...,...,...,...
2671,25.0,2016-09-02 11:20:00,992.0
2672,50.0,2017-11-02 12:17:06,993.0
2673,1000.0,2016-09-13 21:09:47,995.0
2674,1000.0,2017-09-29 20:03:01,995.0


In [47]:
YearJoined = pd.read_csv('data/year_joined.csv')
YearJoined.columns = ['memberId', 'memberStats', 'yearJoined']
YearJoined




,memberId,memberStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018
...,...,...,...
995,995,bronze,2016
996,996,bronze,2018
997,997,bronze,2018
998,998,bronze,2017


In [48]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.member)))

all_email = (emails.set_index(['week', 'member'])
                   .reindex(complete_idx, fill_value=0)
                   .reset_index())
all_email.columns = ['week', 'member', 'emailsOpened']

all_email[all_email.member==998].sort_values(by='week')

,week,member,emailsOpened
81388,2015-02-09,998.0,0.0
58211,2015-02-16,998.0,0.0
58750,2015-02-23,998.0,0.0
65757,2015-03-02,998.0,0.0
72225,2015-03-09,998.0,0.0
...,...,...,...
15091,2018-04-30,998.0,3.0
45275,2018-05-07,998.0,3.0
23176,2018-05-14,998.0,3.0
14552,2018-05-21,998.0,3.0


In [49]:
cutoff_dates = emails.groupby('member').week.agg(['min', 'max']).reset_index()

for _, row in cutoff_dates.iterrows():
    member = row['member']
    start_date = row['min']
    end_date = row['max']
    query = 'member == @member and (week < @start_date or week > @end_date)'
    all_email.drop(all_email.query(query).index, axis='index', inplace=True)

all_email

,week,member,emailsOpened
0,2016-12-05,1.0,3.0
3,2016-12-05,6.0,1.0
4,2016-12-05,9.0,3.0
6,2016-12-05,14.0,0.0
8,2016-12-05,20.0,1.0
...,...,...,...
93231,2017-05-08,959.0,2.0
93236,2017-05-08,970.0,1.0
93237,2017-05-08,973.0,3.0
93241,2017-05-08,987.0,3.0


# Constructing a found time series

In [50]:
# Constructing a found time series
donations.set_index('timestamp', inplace=True)
agg_don = (donations.groupby('member')
                    .apply(lambda df: df.amount.resample('W-MON').sum()))

agg_don = agg_don[agg_don != 0]
agg_don = agg_don.reset_index().set_index('timestamp')   
agg_don

,member,amount
timestamp,,
2015-03-30,0.0,25.0
2015-08-31,0.0,50.0
2016-07-11,0.0,50.0
2017-11-13,0.0,25.0
2016-05-09,1.0,50.0
...,...,...
2018-05-21,992.0,50.0
2017-11-06,993.0,50.0
2016-09-19,995.0,1000.0


In [51]:
lst = []                                                                        
for member, member_email in all_email.groupby('member'):                        
    member_donations = agg_don.query('member == @member')                     
                                                                                
    member_email.set_index('week', inplace=True)                                
    member_email.sort_index(inplace=True)                                       
                                                                                
    df = pd.merge(member_email, member_donations,                               
                  how='left',                                                   
                  left_index=True, right_index=True)                            
    df.fillna(0, inplace=True)                                                  
    df['member'] = df.member_x                                                  
    lst.append(df.reset_index()[['member', 'week', 'emailsOpened', 'amount']])  
                                                                                
merged_df = pd.concat(lst).set_index('week') 
merged_df

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,member,emailsOpened,amount
week,,,
2015-06-29,1.0,3.0,0.0
2015-07-06,1.0,0.0,0.0
2015-07-13,1.0,2.0,0.0
2015-07-20,1.0,2.0,0.0
2015-07-27,1.0,3.0,0.0
...,...,...,...
2018-04-30,998.0,3.0,0.0
2018-05-07,998.0,3.0,0.0
2018-05-14,998.0,3.0,0.0


In [52]:
df = merged_df.query('member == 998')
df

,member,emailsOpened,amount
week,,,
2017-12-04,998.0,1.0,0.0
2017-12-11,998.0,3.0,0.0
2017-12-18,998.0,3.0,0.0
2017-12-25,998.0,0.0,0.0
2018-01-01,998.0,3.0,0.0
2018-01-08,998.0,3.0,50.0
2018-01-15,998.0,2.0,0.0
2018-01-22,998.0,3.0,0.0
2018-01-29,998.0,2.0,0.0


In [53]:
df['target'] = df.amount.shift(1)                                               
df   

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,member,emailsOpened,amount,target
week,,,,
2017-12-04,998.0,1.0,0.0,NaN
2017-12-11,998.0,3.0,0.0,0.0
2017-12-18,998.0,3.0,0.0,0.0
2017-12-25,998.0,0.0,0.0,0.0
2018-01-01,998.0,3.0,0.0,0.0
2018-01-08,998.0,3.0,50.0,0.0
2018-01-15,998.0,2.0,0.0,50.0
2018-01-22,998.0,3.0,0.0,0.0
2018-01-29,998.0,2.0,0.0,0.0
